In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED, DATABASE_DIR
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file, TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [2]:
# cluster
# cluster_netgen
# compression
# expansion
# explosion
# grid
# implosion
# linearprojection
# rotation
# uniform_portgen

In [3]:
N = 30
instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json")

number_of_instances = N // 5
train_instances = InstanceList()
test_instances = InstanceList()
for i in range(10):
    test_instances.append(instances[i * 50])
for i in range(1, 6):
    train_instances.extend(instances[i * 50 + 1: i * 50 + number_of_instances + 1])

In [4]:
for instance in train_instances:
    instance.log()

[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/01.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/02.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/03.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/04.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/05.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/06.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/compression/01.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/compression/02.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/compression/03.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/compression/04.tsp)
[2025-01-01 00:18:57] DEBUG     TSP_Instance(filepat

In [5]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=2)
portfolio

Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]

In [6]:
portfolio.evaluate(test_instances, "test", calculate_features=True)

[2025-01-01 00:18:57] DEBUG     Portfolio.evaluate(test)
[2025-01-01 00:18:57] DEBUG     Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]
[2025-01-01 00:18:57] DEBUG     solve(prefix=test, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/00.tsp))
[2025-01-01 00:18:57] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/00.tsp))


EXECUTOR


[2025-01-01 00:19:05] DEBUG     solve(prefix=test, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/00.tsp))
[2025-01-01 00:19:05] DEBUG     solve(prefix=test, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/00.tsp))
[2025-01-01 00:19:05] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/00.tsp))


EXECUTOR


[2025-01-01 00:19:14] DEBUG     solve(prefix=test, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/00.tsp))
[2025-01-01 00:19:14] DEBUG     solve(prefix=test, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/00.tsp))
[2025-01-01 00:19:14] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/00.tsp))


...

EXECUTOR


[2025-01-01 00:19:59] DEBUG     Solver.Result(prefix=test, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/implosion/00.tsp), cost=100.00, time=14.03, cached=False, surrogate=False, error=False)
[2025-01-01 00:20:01] DEBUG     solve(prefix=test, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/00.tsp))
[2025-01-01 00:20:01] DEBUG     solve(prefix=test, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/uniform_portgen/00.tsp))
[2025-01-01 00:20:01] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/uniform_portgen/00.tsp))


EXECUTOR


[2025-01-01 00:20:04] DEBUG     Solver.Result(prefix=test, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/implosion/00.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)
[2025-01-01 00:20:11] DEBUG     Solver.Result(prefix=test, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/linearprojection/00.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)
[2025-01-01 00:20:11] DEBUG     Solver.Result(prefix=test, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/00.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)
[2025-01-01 00:20:11] DEBUG     Solver.Result(prefix=test, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/00.tsp), cost=100.00, time=13.68, cached=False, surrogate=False, error=False)
[2025-01-01 00:20:13] DEBUG     Solver.Result(prefix=tes

Portfolio.Result(prefix=test, cost=100.00, time=[160.10 100.00])

In [6]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    max_iter=3,
    calculate_features=False,
    estimator=None,
)
aac.configure()

[2024-12-31 23:06:18] DEBUG     AAC(prefix=config, iter=1, configuration_time=[inf inf])
[2024-12-31 23:06:18] DEBUG     Portfolio.evaluate(config;aac_iter=1)
[2024-12-31 23:06:18] DEBUG     Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]
[2024-12-31 23:06:18] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/01.tsp))
[2024-12-31 23:06:18] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/01.tsp))
[2024-12-31 23:06:18] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/01.tsp))
[2024-12-31 23:06:18] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/01.tsp))
[2024-12-31 23:06:18] DEBUG     so

Portfolio(size=2)[Solver(id=2169120517035206132), Solver(id=298144392239699356)]

In [7]:
db = DB()
# db = DB("database/run-plain-50-924800.db")

In [8]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum
0,822075595471039179,TSP/CEPS_benchmark/cluster_netgen/01.tsp,9080004.0
1,213562702282203047,TSP/CEPS_benchmark/compression/01.tsp,16951381.0
2,1184394949679138939,TSP/CEPS_benchmark/expansion/01.tsp,11220460.0
3,1878767591769779070,TSP/CEPS_benchmark/explosion/01.tsp,14162194.0
4,398943848567586880,TSP/CEPS_benchmark/grid/01.tsp,21281520.0


In [9]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.00,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,473831520656465062,0.25,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.50,...,0.470388,0.888889,1.00,0.6,0.2,0.244898,0.0,1.0,3.0,1.0
2,1724561038518932584,0.30,0.0,1.0,2.0,0.6,0.0,1.0,1.0,0.50,...,0.192348,0.777778,0.50,0.4,0.4,0.571429,0.0,1.0,5.0,0.0
3,388564707472054326,0.65,1.0,0.0,3.0,0.8,0.0,0.0,1.0,1.00,...,0.547346,0.222222,0.50,0.8,0.0,0.377551,1.0,0.0,2.0,0.0
4,2169120517035206132,0.35,1.0,1.0,1.0,0.7,0.0,0.0,0.0,0.00,...,0.197133,0.333333,0.25,1.0,1.0,0.510204,1.0,1.0,5.0,0.0
5,298144392239699356,0.30,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.75,...,0.573491,0.222222,0.75,0.6,0.8,0.122449,0.0,0.0,4.0,1.0


In [10]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,config;aac_iter=1_473831520656465062_213562702...,config;aac_iter=1,473831520656465062,213562702282203047,100.00,10.00,0,0,NaN
1,config;aac_iter=1_52980316141715150_8220755954...,config;aac_iter=1,52980316141715150,822075595471039179,100.00,10.00,0,0,NaN
2,config;aac_iter=1_473831520656465062_822075595...,config;aac_iter=1,473831520656465062,822075595471039179,100.00,10.00,0,0,1.0
3,config;aac_iter=1_52980316141715150_2135627022...,config;aac_iter=1,52980316141715150,213562702282203047,100.00,10.00,0,0,1.0
4,config;aac_iter=1_52980316141715150_1184394949...,config;aac_iter=1,52980316141715150,1184394949679138939,100.00,10.00,0,0,1.0
5,config;aac_iter=1_52980316141715150_1878767591...,config;aac_iter=1,52980316141715150,1878767591769779070,100.00,10.00,0,0,NaN
6,config;aac_iter=1_473831520656465062_398943848...,config;aac_iter=1,473831520656465062,398943848567586880,8.84,8.84,0,0,NaN
7,config;aac_iter=1_473831520656465062_118439494...,config;aac_iter=1,473831520656465062,1184394949679138939,100.00,10.00,0,0,1.0
8,config;aac_iter=1_52980316141715150_3989438485...,config;aac_iter=1,52980316141715150,398943848567586880,100.00,10.00,0,0,NaN
9,config;aac_iter=1_473831520656465062_187876759...,config;aac_iter=1,473831520656465062,1878767591769779070,100.00,10.00,0,0,NaN


In [8]:
results.loc[results["prefix"].str.startswith("test")].groupby(["instance_id", "prefix"])["cost"].min().reset_index().head(10)

,instance_id,prefix,cost
0,1018017649469757748,test0,2.08
1,1018017649469757748,test1,100.00
2,1018017649469757748,test2,7.70
3,1018017649469757748,test3,3.36
4,1018017649469757748,test4,100.00
5,1018017649469757748,test5,100.00
6,1018017649469757748,test6,100.00
7,1018017649469757748,test7,5.13
8,1018017649469757748,test8,100.00
9,1018017649469757748,test9,7.00


In [9]:
results.loc[results["prefix"].str.startswith("test")].groupby(["instance_id", "prefix"])["cost"].min().reset_index().sort_values(["instance_id", "cost"]).groupby("instance_id")["cost"].nth(4)

instance_id
1018017649469757748    7.70
1239352391178630764    3.87
146957612122462024     3.75
1765803814609198146    3.32
2137644624762416652    1.59
244685964131091250     0.50
310544774750200091     1.88
396111139638522607     0.54
648058674037521772     7.49
730618644385552802     0.58
Name: cost, dtype: float64

In [25]:
frames = []

for idx, db_path in enumerate(DATABASE_DIR.glob("*.db")):
    db = DB(db_path)
    results = pd.read_sql_query("SELECT * FROM results", db._conn)
    s = (
        results.loc[results["prefix"].str.startswith("test")]
        .groupby(["instance_id", "prefix"])["cost"]
        .min()
        .reset_index()
        .sort_values(["instance_id", "cost"])
        .groupby("instance_id")["cost"]
        .nth(4)
        .rename(idx)
    )
    frames.append(s)
pd.concat(frames, axis=1).mean(axis=1)

instance_id
1018017649469757748    14.913
1239352391178630764    12.171
146957612122462024      3.553
1765803814609198146    52.948
2137644624762416652     2.263
244685964131091250      0.786
310544774750200091      1.844
396111139638522607      0.493
648058674037521772     23.508
730618644385552802      1.138
dtype: float64

In [11]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)

,id,solver_id,instance_id,cost
0,473831520656465062_213562702282203047,473831520656465062,213562702282203047,100.00
1,52980316141715150_822075595471039179,52980316141715150,822075595471039179,100.00
2,473831520656465062_822075595471039179,473831520656465062,822075595471039179,100.00
3,52980316141715150_213562702282203047,52980316141715150,213562702282203047,100.00
4,52980316141715150_1184394949679138939,52980316141715150,1184394949679138939,100.00
5,52980316141715150_1878767591769779070,52980316141715150,1878767591769779070,100.00
6,473831520656465062_398943848567586880,473831520656465062,398943848567586880,8.84
7,473831520656465062_1184394949679138939,473831520656465062,1184394949679138939,100.00
8,52980316141715150_398943848567586880,52980316141715150,398943848567586880,100.00
9,473831520656465062_1878767591769779070,473831520656465062,1878767591769779070,100.00
